In [1]:
import tensorflow as tf
import numpy as np
import os

data_folder = "/home/r.wang/ruowen_git/deepsense/Parse_Raw_HHAR/train_time_24_by_96_slide_6/"
users = "abcdefghi"

In [46]:
test_var_assign = None

def read_csv(user):
    global test_var_assign
    with tf.variable_scope("read_csv", reuse=tf.AUTO_REUSE):
        train_folder = os.path.join(data_folder, user, "train")
        csv_file_list = os.listdir(train_folder)
        filename_queue = tf.train.string_input_producer(
            map(lambda x: os.path.join(train_folder, x), csv_file_list), shuffle=False)
        reader = tf.TextLineReader()
        key, value = reader.read(filename_queue)
        value_in_csv = tf.decode_csv(value, [[0.] for _ in range(24 * 96 + 9)])
        features = tf.reshape(value_in_csv[:24*96], [24, 96])
        label = value_in_csv[24*96:]
        test_var = tf.get_variable("test_var", shape=[2,2], initializer=tf.zeros_initializer())
        test_var_assign = tf.assign(test_var, test_var * 10, name="assign")

        return features, label, test_var

# with tf.device("/cpu:0"):
features, label, test_var = read_csv("a")
print(test_var_assign)

Tensor("read_csv_63/assign:0", shape=(2, 2), dtype=float32_ref)


In [49]:
with tf.variable_scope(None, reuse=True):
    v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
#     print(sess.run(test_var))
    with tf.variable_scope("read_csv", reuse=True):
        print(tf.get_variable("test_var", [2,2]).eval())
    sess.run(test_var.initializer)
    with tf.variable_scope("read_csv", reuse=True):
        print(tf.get_variable("test_var", [2,2]).eval())
#     print(sess.run(test_var_assign))
#     with tf.variable_scope("read_csv", reuse=True):
#         print(tf.get_variable("test_var", [2,2]).read_value().eval())
#     print(tf.convert_to_tensor(a_features))
#     print(a_label)
#     tf.get_variable("read_csv/value_in_csv")
    
    print(v.eval())
    
    coord.request_stop()
    coord.join(threads)

TypeError: If default_name is None then name_or_scope is required